# Hybrid pipeline (Option 2) — Full research-grade notebook

The notebook below contains a full research-grade implementation of the hybrid pipeline. It is split into multiple code cells to avoid parsing issues and to make it easy to run step-by-step. Run on Google Colab or QBraid. Set RUN_MODE='hardware' and configure Qiskit Runtime to run on IBM backends.

In [ ]:
!pip install --quiet qiskit qiskit-aer qiskit-ibm-runtime torch scipy numpy matplotlib nbformat

In [ ]:
import os, math, time, json
import numpy as np
import torch
from torch import nn
from scipy.linalg import expm, eig, fractional_matrix_power
from scipy.signal import find_peaks
import matplotlib.pyplot as plt

# Qiskit imports
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Operator, SparsePauliOp
from qiskit.primitives import Estimator, Sampler
from qiskit.extensions import UnitaryGate
from qiskit.algorithms.optimizers import COBYLA

# Runtime imports (only used in hardware mode)
try:
    from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as RuntimeEstimator, SamplerV2 as RuntimeSampler
except Exception:
    QiskitRuntimeService = None
    RuntimeEstimator = None
    RuntimeSampler = None

print('Imports OK')

In [ ]:
# ------------- CONFIG -------------
RUN_MODE = 'local'   # 'local' or 'hardware'
SAVE_DIR = "results"
os.makedirs(SAVE_DIR, exist_ok=True)

# Hamiltonian truncation / qubits
N_TRUNC = 8           # Fock truncation (tune)
N_QUBITS = int(np.ceil(np.log2(N_TRUNC)))
DIM = 2 ** N_QUBITS

# KAN active learning parameters
KAN_INITIAL = 20
KAN_ITERS = 4
KAN_CAND_PER_ITER = 8
KAN_EPOCHS = 200
KAN_M_UNITS = max(40, 6 * (2 * N_QUBITS))

# VQE refine
VQE_REFINES = 3
VQE_REFINED_ITERS = 120

# Krylov
KRYLOV_DIM = 5

# FFT/time-series
T_MAX = 6.0
N_TIME = 256
DELTA_T = T_MAX / N_TIME

# QPE
QPE_COUNT_QUBITS = 6
QPE_SHOTS = 1024

# Hardware runtime settings (edit if RUN_MODE=='hardware')
RUNTIME_BACKEND_NAME = "ibm_perth"   # replace with your available backend
TROTTER_STEPS = 4

# Random seeds
SEED = 123
np.random.seed(SEED)
torch.manual_seed(SEED)

print('Config set')

In [ ]:
# ------------- Utilities: build H, ladder ops -------------
def ladder_ops(N):
    a = np.zeros((N, N), dtype=complex)
    for n in range(N-1):
        a[n, n+1] = np.sqrt(n+1)
    return a, a.conj().T

def build_H_small(N_trunc):
    a, adag = ladder_ops(N_trunc)
    x = (a + adag) / np.sqrt(2)
    p = 1j * (adag - a) / np.sqrt(2)
    H = 0.5 * (x @ p + p @ x)
    H = 0.5 * (H + H.conj().T)
    return H

H_small = build_H_small(N_TRUNC)
# pad H to DIM
H_pad = np.zeros((DIM, DIM), dtype=complex)
H_pad[:N_TRUNC, :N_TRUNC] = H_small
H_op = Operator(H_pad)

# classical reference
cref_vals, cref_vecs = eig(H_small)
cref_vals = np.real_if_close(cref_vals)
cref_idx = np.argsort(cref_vals)
cref_vals = cref_vals[cref_idx]
print('Built H_small and H_pad. Lowest classical eigenvalues:', np.round(cref_vals[:8],6))

In [ ]:
# ------------- Qiskit runtime or local primitives -------------
if RUN_MODE == 'local':
    aer_sim = AerSimulator(method='statevector')
    estimator = Estimator(aer_sim)
    sampler = Sampler(aer_sim)
    print('RUN_MODE=local: using AerSimulator (statevector)')
else:
    if QiskitRuntimeService is None:
        raise RuntimeError('qiskit_ibm_runtime not available. Install qiskit-ibm-runtime and set RUN_MODE="hardware" only after configuring account.')
    service = QiskitRuntimeService()
    runtime_backend = service.backend(RUNTIME_BACKEND_NAME)
    estimator = RuntimeEstimator(session=service, backend=runtime_backend)
    sampler = RuntimeSampler(session=service, backend=runtime_backend)
    print(f'RUN_MODE=hardware: using runtime backend {RUNTIME_BACKEND_NAME}')

In [ ]:
# ------------- Ansatz and energy evaluation -------------
n_params = 2 * N_QUBITS

def ansatz_circuit(theta):
    qc = QuantumCircuit(N_QUBITS)
    for i in range(N_QUBITS):
        qc.ry(float(theta[i]), i)
        qc.rz(float(theta[i + N_QUBITS]), i)
    for i in range(N_QUBITS - 1):
        qc.cx(i, i+1)
    return qc

def eval_energy_theta(theta):
    qc = ansatz_circuit(theta)
    job = estimator.run([qc], [H_op]).result()
    val = float(job.values[0].real)
    return val

print('Ansatz & energy evaluator ready')

In [ ]:
# ------------- KAN surrogate (PyTorch practical variant) -------------
class KANSurrogate(nn.Module):
    def __init__(self, in_dim, m_units=80, uni_hidden=12):
        super().__init__()
        self.inner = nn.Linear(in_dim, m_units)
        self.uni_w = nn.Parameter(torch.randn(m_units, uni_hidden) * 0.01)
        self.uni_b = nn.Parameter(torch.zeros(m_units, uni_hidden))
        self.uni_out = nn.Parameter(torch.randn(m_units) * 0.01)
        self.outer = nn.Linear(m_units, 1)
    def forward(self, x):
        z = self.inner(x)
        z_exp = z.unsqueeze(-1)
        hidden = torch.tanh(z_exp * self.uni_w.unsqueeze(0) + self.uni_b.unsqueeze(0))
        hid_mean = hidden.mean(dim=-1)
        uni = hid_mean * self.uni_out.unsqueeze(0)
        out = self.outer(uni)
        return out.squeeze(-1)

def train_kan(model, X, y, epochs=150, lr=1e-3):
    model.train()
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    X_t = torch.tensor(X, dtype=torch.float32)
    y_t = torch.tensor(y, dtype=torch.float32)
    for _ in range(epochs):
        opt.zero_grad()
        pred = model(X_t)
        loss = torch.mean((pred - y_t)**2)
        loss.backward()
        opt.step()

def find_minima_kan(model, restarts=8, steps=120, lr=0.2):
    model.eval()
    candidates = []
    for r in range(restarts):
        theta_var = torch.tensor(np.random.uniform(-np.pi, np.pi, size=(1, n_params)), dtype=torch.float32, requires_grad=True)
        opt_x = torch.optim.Adam([theta_var], lr=lr)
        for s in range(steps):
            opt_x.zero_grad()
            pred = model(theta_var)
            pred.backward()
            opt_x.step()
            with torch.no_grad():
                theta_var.clamp_(-4*np.pi, 4*np.pi)
        candidates.append(theta_var.detach().cpu().numpy().reshape(-1))
    return candidates

print('KAN surrogate ready')

In [ ]:
# ------------- Active KAN sampling -------------
def run_kan_active():
    Theta = []
    Evals = []
    print('\nCollecting KAN initial random samples...')
    for i in range(KAN_INITIAL):
        th = np.random.uniform(-np.pi, np.pi, size=(n_params,))
        e = eval_energy_theta(th)
        Theta.append(th); Evals.append(e)
        if i % 5 == 0:
            print(f'  init sample {i}: energy {e:.6f}')
    Theta = np.array(Theta); Evals = np.array(Evals)
    kan = KANSurrogate(n_params, m_units=KAN_M_UNITS).to('cpu')
    for it in range(KAN_ITERS):
        print(f'\nKAN iter {it+1}/{KAN_ITERS}, dataset size {len(Theta)}')
        train_kan(kan, Theta, Evals, epochs=KAN_EPOCHS)
        candidates = find_minima_kan(kan, restarts=KAN_CAND_PER_ITER)
        for j, cand in enumerate(candidates):
            e = eval_energy_theta(cand)
            Theta = np.vstack([Theta, cand]); Evals = np.hstack([Evals, e])
            print(f'  cand {j}: energy {e:.6f}')
        # exploration
        for _ in range(2):
            rth = np.random.uniform(-np.pi, np.pi, size=(n_params,))
            rv = eval_energy_theta(rth)
            Theta = np.vstack([Theta, rth]); Evals = np.hstack([Evals, rv])
            print(f'  random added energy {rv:.6f}')
    best_idx = int(np.argmin(Evals))
    print(f'\nKAN best sampled energy: {Evals[best_idx]:.6f}')
    return Theta, Evals, kan

In [ ]:
# ------------- Short VQE local refinement -------------
def run_vqe_refine(Theta, Evals):
    print('\nRunning short VQE refinements (COBYLA on Estimator primitive)...')
    seed_idxs = np.argsort(Evals)[:VQE_REFINES]
    refined_thetas = []
    refined_energies = []
    for idx in seed_idxs:
        init = Theta[idx]
        def obj(x): return eval_energy_theta(x)
        cobyla = COBYLA(maxiter=VQE_REFINED_ITERS)
        xopt, val, _ = cobyla.optimize(num_vars=n_params, objective_function=obj, initial_point=init)
        refined_thetas.append(xopt); refined_energies.append(val)
        print(f'  seed {idx} refined energy {val:.6f}')
    return refined_thetas, refined_energies

In [ ]:
# ------------- Krylov block construction (simulated on local) -------------
def build_krylov_blocks(refined_thetas):
    print('\nBuilding Krylov blocks (classical H multiplication for demo).')
    krylov_states = []
    for theta in refined_thetas:
        qc = ansatz_circuit(theta)
        sv = aer_sim.run(qc).result().get_statevector(qc) if RUN_MODE == 'local' else None
        if RUN_MODE == 'local':
            v0 = sv[:N_TRUNC]; v0 = v0 / np.linalg.norm(v0)
        else:
            # hardware mode: you'd get amplitude vector from tomography or prepare basis states
            raise NotImplementedError('Krylov classical multiplication in hardware mode requires different strategy.')
        block = [v0]
        for k in range(1, KRYLOV_DIM):
            v = H_small @ block[-1]
            # orthonormalize
            for prev in block:
                v = v - np.vdot(prev, v) * prev
            normv = np.linalg.norm(v)
            if normv < 1e-12:
                break
            block.append(v / normv)
        for vec in block:
            emb = np.zeros(DIM, dtype=complex); emb[:N_TRUNC] = vec; emb /= np.linalg.norm(emb)
            krylov_states.append(emb)
    K = len(krylov_states)
    print(f'  total krylov vectors: {K}')
    # compute S and Hproj (simulated)
    S = np.zeros((K, K), dtype=complex); Hproj = np.zeros((K, K), dtype=complex)
    for i in range(K):
        for j in range(K):
            S[i, j] = np.vdot(krylov_states[i][:N_TRUNC], krylov_states[j][:N_TRUNC])
            Hproj[i, j] = np.vdot(krylov_states[i][:N_TRUNC], H_small @ krylov_states[j][:N_TRUNC])
    S_inv_sqrt = fractional_matrix_power(S, -0.5)
    H_eff = S_inv_sqrt @ Hproj @ S_inv_sqrt
    eigvals_k, eigvecs_k = eig(H_eff)
    eigvals_k = np.real_if_close(eigvals_k)
    ordk = np.argsort(eigvals_k)
    eigvals_k = eigvals_k[ordk]
    print('Krylov-projected eigenvalues (preview):', np.round(eigvals_k[:min(12,len(eigvals_k))],6))
    return krylov_states, S, Hproj, eigvals_k, eigvecs_k, S_inv_sqrt

In [ ]:
# ------------- FFT spectral reconstruction from short-time dynamics -------------
def compute_Ct_via_statevector(psi0, U_step, n_time=N_TIME):
    # psi0: full-dim statevector
    C = np.zeros(n_time, dtype=complex)
    cur = psi0.copy()
    for n in range(n_time):
        C[n] = np.vdot(psi0, cur)
        cur = U_step @ cur
    return C

# Hadamard test builder for hardware
def build_hadamard_test(prep_qc: QuantumCircuit, unitary_circ: QuantumCircuit, measure_imag=False):
    qc = QuantumCircuit(1 + N_QUBITS, 1)
    qc.h(0)
    if measure_imag:
        qc.sdg(0)
    qc.compose(prep_qc, qubits=list(range(1, 1 + N_QUBITS)), inplace=True)
    Ug = unitary_circ.to_gate().control(1)
    qc.append(Ug, [0] + list(range(1, 1 + N_QUBITS)))
    qc.h(0)
    qc.measure(0, 0)
    return qc

print('FFT and Hadamard-test builders ready')

In [ ]:
# ------------- Trotterized controlled evolution builder (hardware-ready) -------------
def pauli_decompose_matrix_to_sparse_pauli(H_mat):
    basis = ['I','X','Y','Z']
    pauli_terms = []
    for idx in range(4 ** N_QUBITS):
        label = ''
        tmp = idx
        for _ in range(N_QUBITS):
            label = basis[tmp % 4] + label
            tmp //= 4
        sp = SparsePauliOp.from_list([(label, 1.0)])
        mat = sp.to_matrix()
        c = np.trace(mat.conj().T @ H_mat) / (2 ** N_QUBITS)
        if abs(c) > 1e-12:
            pauli_terms.append((label, float(c)))
    return pauli_terms

def pauli_term_to_exp_circ(label, angle):
    qc = QuantumCircuit(N_QUBITS)
    for q, p in enumerate(reversed(label)):
        if p == 'X':
            qc.h(q)
        elif p == 'Y':
            qc.sdg(q); qc.h(q)
    for q in range(1, N_QUBITS):
        qc.cx(q, 0)
    qc.rz(2*angle, 0)
    for q in range(1, N_QUBITS):
        qc.cx(q, 0)
    for q, p in enumerate(reversed(label)):
        if p == 'X':
            qc.h(q)
        elif p == 'Y':
            qc.h(q); qc.s(q)
    return qc

def trotterized_controlled_evolution(pauli_terms, t, r_steps):
    qc = QuantumCircuit(1 + N_QUBITS)
    dt = t / r_steps
    for _ in range(r_steps):
        for label, coeff in pauli_terms:
            angle = coeff * dt
            base = pauli_term_to_exp_circ(label, angle)
            gate = base.to_gate().control(1)
            qc.append(gate, [0] + list(range(1, 1 + N_QUBITS)))
    return qc

print('Trotter builder ready')

In [ ]:
# ------------- QPE helpers (QFT dagger) -------------
def qft_dagger(circ, n):
    for i in range(n//2):
        circ.swap(i, n-1-i)
    for j in range(n):
        for m in range(j):
            circ.cp(-np.pi/(2**(j-m)), m, j)
        circ.h(j)
print('QFT dagger ready')

In [ ]:
# ------------- Orchestration: Full pipeline wrappers -------------
def run_kan_active_wrapper():
    Theta, Evals, kan = run_kan_active()
    return Theta, Evals, kan

def run_vqe_refine_wrapper(Theta, Evals):
    return run_vqe_refine(Theta, Evals)

def build_krylov_blocks_wrapper(refined_thetas):
    return build_krylov_blocks(refined_thetas)

def fft_scan_wrapper(refined_thetas):
    print('\nPerforming FFT spectral scan using short-time dynamics')
    U_step = expm(-1j * H_pad * DELTA_T) if RUN_MODE == 'local' else None
    results = []
    for i, theta in enumerate(refined_thetas):
        if RUN_MODE == 'local':
            qc = ansatz_circuit(theta)
            sv = aer_sim.run(qc).result().get_statevector(qc)
            C_t = compute_Ct_via_statevector(sv, U_step, n_time=N_TIME)
        else:
            raise NotImplementedError('Hardware FFT measurement path requires scheduled Hadamard-test runs and streaming.')
        window = np.hanning(N_TIME)
        C_w = C_t * window
        spectrum = np.fft.fftshift(np.fft.fft(C_w))
        freqs = np.fft.fftshift(np.fft.fftfreq(N_TIME, d=DELTA_T))
        power = np.abs(spectrum)
        peak_idx, props = find_peaks(power, height=np.max(power)*0.07, distance=2)
        peak_freqs = freqs[peak_idx]
        peak_vals = power[peak_idx]
        sorted_peaks = sorted(zip(peak_vals, peak_freqs), key=lambda x: -x[0])
        print(f'  seed {i}: found {len(sorted_peaks)} peaks. Top freqs:', [round(p[1],6) for p in sorted_peaks[:5]])
        results.append({
            'seed_idx': i, 'time': np.linspace(0, T_MAX, N_TIME, endpoint=False),
            'C_t': C_t, 'freqs': freqs, 'power': power, 'sorted_peaks': sorted_peaks
        })
    return results

def qpe_refine_wrapper(fft_results, refined_thetas, krylov_states, S_inv_sqrt, eigvecs_k):
    print('\nQPE refinement seeded by FFT peaks')
    qpe_estimates = []
    if RUN_MODE == 'local':
        U_total = expm(-1j * H_pad * 1.0)  # t=1
        for seed_i, res in enumerate(fft_results):
            top = res['sorted_peaks'][:6]
            for val, freq in top:
                E_coarse = 2 * np.pi * freq
                qc = ansatz_circuit(refined_thetas[seed_i])
                sv = aer_sim.run(qc).result().get_statevector(qc)
                approx = sv
                qc_qpe = QuantumCircuit(QPE_COUNT_QUBITS + N_QUBITS, QPE_COUNT_QUBITS)
                qc_qpe.initialize(approx.tolist(), list(range(QPE_COUNT_QUBITS, QPE_COUNT_QUBITS + N_QUBITS)))
                for q in range(QPE_COUNT_QUBITS):
                    qc_qpe.h(q)
                for j in range(QPE_COUNT_QUBITS):
                    power = 2 ** (QPE_COUNT_QUBITS - 1 - j)
                    U_pow = np.linalg.matrix_power(U_total, power)
                    qc_qpe.append(UnitaryGate(U_pow).control(1), [j] + list(range(QPE_COUNT_QUBITS, QPE_COUNT_QUBITS + N_QUBITS)))
                qft_dagger(qc_qpe, QPE_COUNT_QUBITS)
                for q in range(QPE_COUNT_QUBITS):
                    qc_qpe.measure(q, q)
                job = aer_sim.run(qc_qpe, shots=QPE_SHOTS).result()
                counts = job.get_counts(qc_qpe)
                most = max(counts.items(), key=lambda kv: kv[1])[0]
                m = int(most, 2)
                phi = m / (2**QPE_COUNT_QUBITS) * 2*np.pi
                E_refined = phi / 1.0
                qpe_estimates.append({'seed': seed_i, 'freq': float(freq), 'coarse_E': float(E_coarse), 'refined_E': float(E_refined)})
                print(f'  seed {seed_i} freq {freq:.6f} -> coarse E {E_coarse:.6f}, QPE refined {E_refined:.6f}')
    else:
        pauli_terms = pauli_decompose_matrix_to_sparse_pauli(H_pad)
        ctrl_evo = trotterized_controlled_evolution(pauli_terms, t=1.0, r_steps=TROTTER_STEPS)
        raise NotImplementedError('Hardware QPE runtime submission not implemented in demo script (requires account).')
    return qpe_estimates

In [ ]:
# ------------- Plotting & saving -------------
def save_and_plot_results(fft_results, qpe_results, krylov_eigs, refined_energies, do_plot=True):
    summary = {
        'fft_summary': [{'seed': r['seed_idx'], 'num_peaks': len(r['sorted_peaks'])} for r in fft_results],
        'qpe_summary': qpe_results,
        'krylov_eigs': [float(x) for x in krylov_eigs[:min(20,len(krylov_eigs))]],
        'refined_energy_seeds': [float(x) for x in refined_energies]
    }
    with open(os.path.join(SAVE_DIR, 'summary.json'), 'w') as f:
        json.dump(summary, f, indent=2)
    for i, r in enumerate(fft_results):
        freqs = r['freqs']; power = r['power']
        plt.figure(figsize=(8,4)); plt.plot(freqs, power); plt.title(f'FFT spectral power seed {i}');
        peaks = [p[1] for p in r['sorted_peaks'][:8]]
        plt.scatter(peaks, [np.max(power)*0.9]*len(peaks), color='red')
        plt.xlabel('Frequency'); plt.ylabel('Power')
        plt.tight_layout()
        plt.savefig(os.path.join(SAVE_DIR, f'fft_seed_{i}.png'))
        if do_plot:
            plt.show()
        plt.close()
    if qpe_results:
        with open(os.path.join(SAVE_DIR, 'qpe_results.json'), 'w') as f:
            json.dump(qpe_results, f, indent=2)
    print(f'\nResults saved to {SAVE_DIR}')

In [ ]:
# ------------- Main entrypoint: run small experiment -------------
if __name__ == '__main__':
    if RUN_MODE == 'local':
        aer_sim = AerSimulator(method='statevector')
        estimator = Estimator(aer_sim)
        Theta, Evals, kan_model = run_kan_active_wrapper()
        refined_thetas, refined_energies = run_vqe_refine_wrapper(Theta, Evals)
        krylov_states, S, Hproj, eigvals_k, eigvecs_k, S_inv_sqrt = build_krylov_blocks_wrapper(refined_thetas)
        fft_results = fft_scan_wrapper(refined_thetas)
        qpe_results = qpe_refine_wrapper(fft_results, refined_thetas, krylov_states, S_inv_sqrt, eigvecs_k)
        save_and_plot_results(fft_results, qpe_results, eigvals_k, refined_energies, do_plot=True)
    else:
        print('Hardware mode selected. Configure QiskitRuntimeService and set RUNTIME_BACKEND_NAME.')